Our project topic:
Back-test a rotation strategy based on aggregate valuations of the NIFTY non-financial sector indices (AUTO, COMMODITIES, FMCG, etc...)

How we have planned to go about it:

1. Taking 5-6 non-financial sector indices like Auto, commodities, metal, fmcg,pharma.

2. Determine seasonality and cyclicity in the each of these chosen indices. 

3. Based on this information, determine the frequency and the factors to be considered for rotation. 

4. Performing 5 different rotation strategies like:
Fixed ratio
Beta based
Variable ratio
Constant rupee plan
Constant ratio plan
This will help us understand the timing of entry and exit and the composition of the portfolio yielding maximum returns. 

5. Comparing the returns form each of these five types of strategies with that of a portfolio without any rotation IE, an untouched portfolio. 

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")


indices<-Indices()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 

Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from

In [2]:
today <- Sys.Date()
startDt <- today - 10

indices$NseTimeSeries() %>%
  filter(TIME_STAMP >= startDt) %>%
  arrange(desc(TIME_STAMP)) %>%
  print()

# Source:     lazy query [?? x 7]
# Database:   Microsoft SQL Server 13.00.4259[ro1@NORWAY/StockViz]
# Ordered by: desc(TIME_STAMP)
   NAME                 TIME_STAMP  HIGH   LOW  OPEN CLOSE    VOLUME
   <chr>                <date>     <dbl> <dbl> <dbl> <dbl>     <dbl>
 1 NIFTY500 VALUE 50 TR 2019-08-16   NA    NA    NA  5299.        NA
 2 NIFTY500 VALUE 50    2019-08-16   NA    NA    NA  3577. 319955857
 3 NIFTY500 SHARIAH TR  2019-08-16   NA    NA    NA  3280.        NA
 4 NIFTY500 SHARIAH     2019-08-16   NA    NA    NA  2797. 204288892
 5 NIFTY50 VALUE 20 TR  2019-08-16   NA    NA    NA  6321.        NA
 6 NIFTY50 VALUE 20     2019-08-16 5224. 5151. 5223  5192. 272052526
 7 NIFTY50 TR 2X LEV    2019-08-16 5812. 5661  5786. 5790         NA
 8 NIFTY50 TR 1X INV    2019-08-16  373.  368.  369.  369.        NA
 9 NIFTY50 SHARIAH TR   2019-08-16   NA    NA    NA  2849.        NA
10 NIFTY50 SHARIAH      2019-08-16   NA    NA    NA  2395.  78765470
# … with more rows


In [ ]:
today <- Sys.Date()
startDt <- today - 100

indices$NseTimeSeries() %>%
  filter(TIME_STAMP >= startDt) %>%
  arrange(desc(TIME_STAMP)) %>%
  print(n=20)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)